In [1]:
%run forward_simulation.ipynb

In [ ]:
uv_control = control(uv_body=uv_body)
pid_control, i_buffer = uv_control.position_pid()
pid_control_reg, i_buffer_reg = uv_control.position_pid_reg()
trackingpid, trac_i_buffer = uv_control.trajectorytracking_pid()

In [ ]:
pid_controller = ca.Function('pid_control', [W, B, r_g, r_b, Kp, Ki, Kd, sum_e_buffer, sim_x, nd, xb_d, dt], [pid_control, i_buffer])
pid_controller_reg = ca.Function('pid_control_reg', [uv_control.lumped_params, Kp, Ki, Kd, sum_e_buffer, sim_x, nd, xb_d, dt], [pid_control_reg, i_buffer_reg])
trackingpid_controller = ca.Function('trackingpid', [uv_control.lumped_params, Kp, Ki, Kd, sum_e_buffer, sim_x, nd, xb_d, des_acc_b, dt, v_c], [trackingpid, trac_i_buffer])
trackingpid_controller_ = trackingpid_controller(beta_sym, Kp, Ki, Kd, sum_e_buffer, sim_x, nd, xb_d, des_acc_b, dt, v_c)

In [ ]:
x0 = np.array([0.0, 0.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) #x y z r p y u v w p q r # initial conditions
xf = np.array([2.0, 0.0, 15.0, 0.0, 0.0, 0.0]) #x y z r p y # goal
sebuffer_ = np.array([0,0,0,0,0,0])
desired_body_vel = np.array([0,0,0,0,0,0])
desired_body_acc = np.array([0,0,0,0,0,0])

pid_controller(blue.W, blue.B, blue.rg, blue.rb, blue.kp, blue.ki, blue.kd, sebuffer_, x0, xf, desired_body_vel, blue.dt_s)

(DM([1, 0, 7.66167, 0, 0, 0]), DM([0.025, 0, 0.175, 0, 0, 0]))

In [ ]:
pid_controller_reg_ = ca.Function('pid_control_reg_test', [sim_p, Kp, Ki, Kd, sum_e_buffer, sim_x, nd, xb_d, dt], pid_controller_reg(beta_sym, Kp, Ki, Kd, sum_e_buffer, sim_x, nd, xb_d, dt))
pid_controller_reg_(blue.sim_params, blue.kp, blue.ki, blue.kd, sebuffer_, x0, xf, desired_body_vel, blue.dt_s)

(DM([1, 0, 7.66167, 0, 0, 0]), DM([0.025, 0, 0.175, 0, 0, 0]))

In [ ]:
_trackingpid_controller = ca.Function('trackingpid_controller_test', [sim_p, Kp, Ki, Kd, sum_e_buffer, sim_x, nd, xb_d, des_acc_b, dt], trackingpid_controller_)
_trackingpid_controller(blue.sim_params, [0.1,0.1,0.1, 0.1,0.1,0.1], blue.ki, [0.1,0.1,0.1,0.1,0.1,0.1], sebuffer_, x0, xf, desired_body_vel, desired_body_acc, blue.dt_s)

(DM([0.34, 0, 4.31147, 0, 0.0046, 0]), DM([0.025, 0, 0.175, 0, 0, 0]))

In [6]:
# # c , cpp or matlab code generation for forward dynamics
# pd_controller.generate("PdController.c")
# os.system(f"gcc -fPIC -shared PdController.c -o libPd.so")